<a href="https://colab.research.google.com/github/LAM-GROUP/PLIP/blob/develop/PLIPwithLAMMPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.Install LAMMPS**


*   Note the warning messages in LAMMPS can be  ignored, since it doesn't effect the production run.



In [5]:
!git clone https://github.com/LAM-GROUP/LammpsWithPLIP.git
%cd LammpsWithPLIP/src
!make -j 8 mpi


Streaming output truncated to the last 5000 lines.
  140 |     fgets(line,MAXLINE,fptr);
      |     ~~~~~^~~~~~~~~~~~~~~~~~~
../pair_lars_NB_18.cpp:141:10: warning: ignoring return value of ‘char* fgets(char*, int, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  141 |     fgets(line,MAXLINE,fptr);
      |     ~~~~~^~~~~~~~~~~~~~~~~~~
../pair_lars_NB_18.cpp:142:10: warning: ignoring return value of ‘char* fgets(char*, int, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  142 |     fgets(line,MAXLINE,fptr);
      |     ~~~~~^~~~~~~~~~~~~~~~~~~
../pair_lars_NB_18.cpp:143:10: warning: ignoring return value of ‘char* fgets(char*, int, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  143 |     fgets(line,MAXLINE,fptr);
      |     ~~~~~^~~~~~~~~~~~~~~~~~~
../pair_lars_NB_18.cpp:168:10: warning: ignoring return value of ‘char* fgets(char*, int, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  168 |  

# **2. Check if LAMMPS is installed properly**

In [11]:
%cd ../../
!LammpsWithPLIP/src/lmp_mpi -h



/content
LAMMPS (11 Aug 2017)

Large-scale Atomic/Molecular Massively Parallel Simulator - 11 Aug 2017

Usage example: LammpsWithPLIP/src/lmp_mpi -var t 300 -echo screen -in in.alloy

List of command line options supported by this LAMMPS executable:

-echo none/screen/log/both  : echoing of input script (-e)
-help                       : print this help message (-h)
-in filename                : read input from file, not stdin (-i)
-kokkos on/off ...          : turn KOKKOS mode on or off (-k)
-log none/filename          : where to send log output (-l)
-nocite                     : disable writing log.cite file (-nc)
-package style ...          : invoke package command (-pk)
-partition size1 size2 ...  : assign partition sizes (-p)
-plog basename              : basename for partition logs (-pl)
-pscreen basename           : basename for partition screens (-ps)
-restart rfile dfile ...    : convert restart to data file (-r)
-reorder topology-specs     : processor reordering (-r)
-screen 

# **3.Install PLIP**

In [12]:
!git clone https://github.com/LAM-GROUP/PLIP.git
%cd PLIP
!pip3 install -e .

Cloning into 'PLIP'...
remote: Enumerating objects: 667, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 667 (delta 24), reused 57 (delta 18), pack-reused 587
Receiving objects: 100% (667/667), 78.62 MiB | 19.90 MiB/s, done.
Resolving deltas: 100% (254/254), done.
Updating files: 100% (232/232), done.
/content/PLIP
Obtaining file:///content/PLIP
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Building editable for plip (pyproject.toml) ... done
  Created wheel for plip: filename=plip-0.3.3-0.editable-py3-none-any.whl size=27158 sha256=b312f25159777b1887a96ac005a188a1ffbfdfd235b9b06db7665a3f020319e8
  Stored in directory: /tmp/pip-ephem-wheel-cache-ws836qur/wheels/00/55/11/181522

# **4.Check if PLIP installed properly**

In [13]:
!plip_fit --help


usage: PLIP
       [-h]
       [--yaml {input.yaml}]
       [--compile]
       [--genbin {input.txt}]
       [--train {1}]
       [--alpha ALPHA]
       [--genpot]
       [--version]

Physical
Lassolars
interaction
potentials

options:
  -h, --help
    show this
    help
    message and
    exit
  --yaml {input.yaml}, -i {input.yaml}
    PLIP
    workflow
  --compile, -c
    Compile C++
    modules
  --genbin {input.txt}, -g {input.txt}
    Generate
    binary
    files
  --train {1}, -t {1}
    Run the
    lassolars
  --alpha ALPHA, -a ALPHA
    Value of
    alpha
  --genpot, -p
    Generate
    potential
    for all
    coefficient
    files
  --version, -v
    version of
    the package


# **5. Two body MLIP fitting for SiO2**





In [15]:
%cd  /content/PLIP/example/1.Twobody_potential_SiO2/2.YAML_interface/


/content/PLIP/example/1.Twobody_potential_SiO2/2.YAML_interface


#  **6.Generate two-body fingerprint**


In [16]:
import yaml


data = {
        "Input": {                          # Inputs for machine learning fingerprint
            "Rcut": 6,                      # Cut-off distance
            "DUMP": "Input",                # Folder location of structure files
            "RAA": 2.1,                     # Nbody cut-off
            "RBB": 2.3,                     # Nbody cut-off
            "RAB": 2.4,                     # Nbody cut-off
            "i_nature": 1,                  # Descriptor type 1: gaussian
            "included": 1,
            "a": {                          # Width parameter gaussian
                "MIN": 0.5,
                "MAX": 1.5,
                "STEP": 0.5},
            "b": {                          # Position parameter gaussian
                "MIN": 0.5,
                "MAX": 6,
                "STEP": 0.5},
            "2B": 1,                        # Generate Twobody descriptor
            "3B": 0,                        # Generate threebody descriptor(if present value=5)
            "NB": 0,                        # Generate threebody descriptor(if present value=7)
        },
        "compile": True,                   # Compile C++ modules
        "genBin": True,                     # Generate machine learning fingerprint
        "train": {                          # Fit LassoLars
            "i_nature": "1",                # Descriptor type 1: gaussian
             "alpha":1e-06
        },
        "genPot":True,                     # Generate LAMMPS potential
    }

with open("input.yaml", "w") as f:
        yaml.dump(data, f, sort_keys=False)

# **Fit the PLIP and generate LAMMPS potential**

In [17]:
!plip_fit -i  input.yaml

Namespace(yaml='input.yaml', compile=False, genbin=None, train=None, alpha=None, genpot=False, version=False)
{'Input': {'Rcut': 6, 'DUMP': 'Input', 'RAA': 2.1, 'RBB': 2.3, 'RAB': 2.4, 'i_nature': 1, 'included': 1, 'a': {'MIN': 0.5, 'MAX': 1.5, 'STEP': 0.5}, 'b': {'MIN': 0.5, 'MAX': 6, 'STEP': 0.5}, '2B': 1, '3B': 0, 'NB': 0}, 'compile': True, 'genBin': True, 'train': {'i_nature': '1', 'alpha': 1e-06}, 'genPot': True}
input.txt file has been generated.
/content/PLIP/src/plip
Compilation of XMAT successful.
/content/PLIP/src/plip
Compilation of GEN_POT successful.

=====================   XMAT   CALCULATIONS   =====================


=====================     INITIALIZATION     =====================
Input_folder=Input
Rcut=6
R_AA=2.1
R_BB=2.3
R_AB=2.4
=====================  Input/213.poscar  =====================
15.8  0  0
0  15.8  0
0  0  15.8
Size=4
Na=128  Nb=256
Distances and angles DONE
Working with Gaus
list_a= 0.5  1  1.5  
list_b= 0.5  1  1.5  2  2.5  3  3.5  4  4.5  5  5.5  6 

sample_data


# **Check PLIP outputs**

In [18]:
!cat Output_1.log

alpha  scoreTrain scoreTest RMSE_Train RMSE_Test MAE_Train MAE_Test numNonZeroCoeff numTotalCoeff meanAbsYTrain
1e-06  0.777573  0.819775  0.347687  0.342279   0.268267   0.268048   40   108   0.513926  


# **Check Potential files are generated**

* The two body tables are kept in out2B_{num}.txt files

In [20]:
!ls POTS/Coeff1e-06_1/

out2B_0.txt  out2B_1.txt  out2B_2.txt


# **Run LAMMPS**
We are going to run a simple mimisation of SiO2 coesite crystal with LAMMPS. All the scripts have been prepared and kept im lammps/ directory

In [22]:
%cd lammps

/content/PLIP/example/1.Twobody_potential_SiO2/2.YAML_interface/lammps


In [23]:
# See the lammps input configuration.
!cat input.lmp

units           metal
dimension       3
boundary        p p p

read_data	IN.data

pair_style  hybrid/overlay table linear 1000

pair_coeff      1 1  table           ../POTS/Coeff1e-06_1/out2B_0.txt LARS_2B 6
pair_coeff      2 2  table           ../POTS/Coeff1e-06_1/out2B_1.txt LARS_2B 6
pair_coeff      1 2  table           ../POTS/Coeff1e-06_1/out2B_2.txt LARS_2B 6


mass            1 28.0855
mass            2 15.999

thermo		10


fix             box1 all box/relax tri 0.0
thermo_style 	custom step cella cellb cellc cellalpha cellbeta cellgamma 
dump            002 all custom 10 dump_xyz.lammpstrj id type x y z fx fy fz
dump_modify     002 sort id

minimize        1.0e-12 1.0e-12 15000 20000

           



In [25]:
!/content/LammpsWithPLIP/src/lmp_mpi -i input.lmp

LAMMPS (11 Aug 2017)
Reading data file ...
  triclinic box = (0 0 0) to (7.15711 12.4208 6.21716) with tilt (2.32723e-06 -3.51342 5.8702e-07)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  Should only be flagged at inflection points (../pair_table.cpp:481)
  Should only be flagged at inflection points (../pair_table.cpp:481)
  Should only be flagged at inflection points (../pair_table.cpp:481)
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 8
  ghost atom cutoff = 8
  binsize = 4, bins = 3 4 2
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair table, perpetual
      attributes: half, newton on
      pair build: half/bin/newton/tri
      stencil: half/bin/3d/newton/tri
      bin: standard
Setting up cg style minimization ...
  Unit style    : metal
  Current step  : 0
Per MPI rank memory allocation (min/avg/max) = 5.132 | 5.132 | 5.132 Mbytes
Step Cella

In [30]:
!tail log.lammps

Histogram: 1 0 0 0 0 0 0 0 0 0

Total # of neighbors = 10356
Ave neighs/atom = 215.75
Neighbor list builds = 4
Dangerous builds = 0



Total wall time: 0:00:01
